# Multiple Linear Regression
### Using Gradient Descent

In [9]:
import numpy as np
import pandas as pd

In [10]:
batch_size = 1000
learning_rate = 0.000001
num_of_epoch = 10000
test_training_ratio = 0.2
# dataset_path = '/content/sample_data/data.csv' # if running in collab
dataset_path = 'data.csv' #if running locally

In [11]:
# Read the dataset
df = pd.read_csv(dataset_path) 
df.head()

,AT,V,AP,RH,PE
0,8.34,40.77,1010.84,90.01,480.48
1,23.64,58.49,1011.40,74.20,445.75
2,29.74,56.90,1007.15,41.91,438.76
3,19.07,49.69,1007.22,76.79,453.09
4,11.80,40.66,1017.13,97.20,464.43


In [12]:
# Total_X is a matrix the size of 9568 X 4
total_X = df.iloc[:, :-1].values 

# Total_Y is a vector the size of 9568
total_y = df.iloc[:, -1].values  

In [13]:
b = np.ones((total_X.shape[0],total_X.shape[1]+1)) # b is a 9568 X 5 matrix filled with ones.
b[:, 1:] = total_X #b is the total_X matrix except with first column filled with ones
total_X = b #reassign
print(total_X[:5])

[[1.00000e+00 8.34000e+00 4.07700e+01 1.01084e+03 9.00100e+01]
 [1.00000e+00 2.36400e+01 5.84900e+01 1.01140e+03 7.42000e+01]
 [1.00000e+00 2.97400e+01 5.69000e+01 1.00715e+03 4.19100e+01]
 [1.00000e+00 1.90700e+01 4.96900e+01 1.00722e+03 7.67900e+01]
 [1.00000e+00 1.18000e+01 4.06600e+01 1.01713e+03 9.72000e+01]]


In order for us to verify the accuracy later, we will reserve about 20% of the whole dataset.

In [14]:
num_of_testing_example    = int(test_training_ratio*total_X.shape[0]) # (20/100) * 9568 = 1913.6. After flooring, 1913
num_of_training_example   = total_X.shape[0] - num_of_testing_example # 9568 - 1913 = 7655
X                         = total_X[:num_of_training_example] #matrix of size 7655 X 4. input X (according to the pdf note)
y                         = total_y[:num_of_training_example] #vector of size 7655. input y (according to the pdf note)
X_test                    = total_X[num_of_training_example:] #matrix of size 1913 X 4
y_test                    = total_y[num_of_training_example:] #vector of size 1913

In [15]:
theta = np.random.rand((X.shape[1])) #randomly initialize theta as a vector of size 5

Parameters :

    data_point : Batches of rows in the training set matrix
    theta: theta vector

    Process : matrix (data points) multiplies vector (theta) 

    Output : prediction of the model, y_hat or h(X)

In [16]:
def find_y_hat(data_point,theta):
    return np.dot(data_point, theta) 

In [17]:
def testing_with_MSE():
    
    diff = X_test.dot(theta) - y_test #calculate (h(x) - y)
    
    loss = (1/(2*X_test.shape[0]))*(diff.dot(diff)) #calculate the loss using the defined equation
    
    print("Testing loss with MSE is : ", loss)
    
def testing_with_R2():
    
    diff = X_test.dot(theta) - y_test #calculates h(x) - y

    u = sum(np.square(diff)) #variance of the data from the prediction model
    v = sum(np.square(y_test - np.mean(y_test))) #variance of the original data

    print("The testing loss with R2 is :", (1 - (u/v))) #print the R2 coefficient


testing_with_MSE() #run the testing with MSE before the training begins
testing_with_R2() #run the testing with R2 before the training begins

Testing loss with MSE is :  303.22668336958947
The testing loss with R2 is : -1.0162202998998908


In [18]:
for epoch in range(num_of_epoch):
    
    diff = X.dot(theta) - y #calculate (h(x) - y)
    
    loss = (1/(2*X.shape[0]))*(diff.dot(diff))  #calculate the loss using the defined equation
    
    for index_first in range(0, X.shape[0], batch_size): #loop through the dataset with the given batch size
        
        index_last = index_first + batch_size #index_first - index_last = batch size
        index_last = None if index_last > X.shape[0] else index_last #if index_last > total no. of data points, it will be set to None
        
        y_hat = find_y_hat(X[index_first:index_last], theta) #predicted output
        
        m = y_hat.shape[0] # number of data points i.e. batch size
        
        y_hat_diff = y_hat - y[index_first:index_last] #calculate (h(x) - y)
        
        
        for j in range(theta.shape[0]): #gradient descent
        
            x_j = X[index_first:index_last, j] #get the x_j vector for that specified batch size
        
            par_der = (1/m)*(y_hat_diff.dot(x_j)) #calculate the partial derivative
            
            theta[j] = theta[j] - learning_rate*par_der
    
    if epoch % 1000 == 0: #print loss for every 500 epoch starting from 0th epoch
        print("Epoch : ", epoch)
        print("Loss : ", loss)
    

Epoch :  0
Loss :  302.97629282240126
Epoch :  1000
Loss :  40.35318992698053
Epoch :  2000
Loss :  26.891062659301348
Epoch :  3000
Loss :  23.996962762746964
Epoch :  4000
Loss :  22.386242074412944
Epoch :  5000
Loss :  21.092088313511443
Epoch :  6000
Loss :  19.983528498107276
Epoch :  7000
Loss :  19.02595613675143
Epoch :  8000
Loss :  18.19804172849076
Epoch :  9000
Loss :  17.482190822012928


In [19]:
print(theta)

[ 0.104257   -0.83328772 -0.66685975  0.49837422  0.02744297]


In [20]:
testing_with_MSE() #as expected gradient descent optimization works slightly worse than the normal equation method.
testing_with_R2()

Testing loss with MSE is :  16.595353598483687
The testing loss with R2 is : 0.8896538772991269


In [21]:
X_test.dot(theta)- y_test

array([-1.85355946, -0.98030388, -2.08395879, ..., -0.47191881,
        4.77332406,  0.73207242])